# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
# Make a reference to the comic_books_expanded.csv file path
csv_path = "../output_data/weather_data.csv"

# Import the comic_books_expanded.csv file as a DataFrame
city_df = pd.read_csv(csv_path, encoding="utf-8")
# Check the special characters imported correctly
city_df.tail()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
563,563,Gorontalo,ID,1666053862,0.5412,123.0595,86.77,65,100,1.12
564,564,Thunder Bay,CA,1666053670,48.4001,-89.3168,30.18,59,40,18.41
565,565,Cruzeiro do Sul,BR,1666053863,-29.5128,-51.9853,67.51,77,100,9.06
566,566,Concordia,AR,1666053863,-31.3930,-58.0209,65.79,72,100,8.01
567,567,Kolyvan',RU,1666053864,55.3053,82.7391,40.62,86,100,10.49


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
gmaps.configure(g_key)

In [31]:
marker_locations = city_df[['Latitude', 'Longitude']]
humidity = city_df["Humidity"]
fig = gmaps.figure()
markers = gmaps.heatmap_layer(marker_locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(city_df["Humidity"]),
                                 point_radius = 2)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
# Narrow down the DataFrame to find your ideal weather condition.
fitting_df = city_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
fitting_df = fitting_df.loc[(fitting_df["Max Temperature"] < 80) & (fitting_df["Max Temperature"] > 70)]

# Wind speed less than 10 mph.
fitting_df = fitting_df.loc[fitting_df["Wind Speed"] < 10]

# Zero cloudiness.
fitting_df = fitting_df.loc[fitting_df["Cloudiness"] == 0]

# Drop any rows with null values
fitting_df = fitting_df.dropna()

fitting_df

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
20,20,Chubbuck,US,1666053709,42.9208,-112.4661,73.29,27,0,3.44
54,54,Aswān,EG,1666053717,24.0934,32.9070,76.30,35,0,5.75
103,103,Altamont,US,1666053731,42.2068,-121.7372,75.04,17,0,0.00
148,148,Fort Abbas,PK,1666053744,29.1925,72.8536,74.66,20,0,9.06
190,190,Fernley,US,1666053756,39.6080,-119.2518,77.86,16,0,4.92
249,249,Hatillo,PR,1666053772,18.4863,-66.8254,78.85,90,0,1.99
251,251,Arinos,BR,1666053772,-15.9169,-46.1056,75.87,32,0,3.00
252,252,Jesup,US,1666053593,31.6074,-81.8854,73.35,78,0,0.00
297,297,Caravelas,BR,1666053785,-17.7125,-39.2481,74.08,83,0,7.02
301,301,Burley,US,1666053786,42.5357,-113.7928,71.56,21,0,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = fitting_df
radius = 5000

params = {
    "radius": radius,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params)
    response_json = response.json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response_json['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 20: Chubbuck.
Closest hotel is Quality Inn.
------------
Retrieving Results for Index 54: Aswān.
Closest hotel is Nuba Nile.
------------
Retrieving Results for Index 103: Altamont.
Closest hotel is Microtel Inn & Suites by Wyndham Klamath Falls.
------------
Retrieving Results for Index 148: Fort Abbas.
Closest hotel is AsAd's House.
------------
Retrieving Results for Index 190: Fernley.
Closest hotel is Super 8 by Wyndham Fernley.
------------
Retrieving Results for Index 249: Hatillo.
Closest hotel is El Buen Hotel.
------------
Retrieving Results for Index 251: Arinos.
Closest hotel is casa do bruno.
------------
Retrieving Results for Index 252: Jesup.
Closest hotel is Western Motel.
------------
Retrieving Results for Index 297: Caravelas.
Closest hotel is Pousada dos Navegantes.
------------
Retrieving Results for Index 301: Burley.
Closest hotel is Fairfield Inn & Suites by Marriott Burley.
------------
Retrieving Results for Index 331: Winnemucca.

KeyboardInterrupt: 

In [37]:
hotel_df.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
20,20,Chubbuck,US,1666053709,42.9208,-112.4661,73.29,27,0,3.44,Quality Inn
54,54,Aswān,EG,1666053717,24.0934,32.9070,76.30,35,0,5.75,Nuba Nile
103,103,Altamont,US,1666053731,42.2068,-121.7372,75.04,17,0,0.00,Microtel Inn & Suites by Wyndham Klamath Falls
148,148,Fort Abbas,PK,1666053744,29.1925,72.8536,74.66,20,0,9.06,AsAd's House
190,190,Fernley,US,1666053756,39.6080,-119.2518,77.86,16,0,4.92,Super 8 by Wyndham Fernley


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [40]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))